In [4]:
import os
os.system("sudo apt-get update")
os.system("sudo apt-get install -y python3-opencv")
os.system("pip install opencv-python")
os.system("pip install matplotlib")
import sys
sys.path.append("..")
import cv2
import numpy as np
from utils.imutils import jimshow as show
from utils.imutils import jimshow_channel as show_channel
import matplotlib.pyplot as plt

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
python3-opencv is already the newest version (4.5.4+dfsg-9ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
#function for reading in a file and normailzing it
def normit(image):
    hist = cv2.calcHist([image],[0,1,2],None,[255,255,255],[0,256,0,256,0,256])
    normalized_hist = cv2.normalize(hist, hist, 0, 1.0, cv2.NORM_MINMAX)
    return(normalized_hist)

# loop load all flowers and names and save it
fp = "/work/cds-vis-data/flowers/"
fp_s = os.listdir(fp)


image_list = []
name_list = []
for i in fp_s:
    filepath = fp + i
    image = cv2.imread(filepath)
    names = i
    name_list.append(i)
    image_list.append(image)





In [8]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [9]:
#loop and normalize flowers 100 at a time
norm_list =[]
for i in chunker(image_list,100):
    norm_one = normit(i)
    norm_list.append(norm_one)

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'calcHist'
> Overload resolution failed:
>  - Can't parse 'images'. Sequence item with index 0 has a wrong type
>  - Can't parse 'images'. Sequence item with index 0 has a wrong type


In [13]:
test_list = []

for i in chunker(image_list[:5],1):
    test_list.append(i)

In [22]:
test_list = []
for i in range(0,len(image_list)):

    for j in image_list[i]:
        test = normit(j)
    
    test_list.append(test)
        

error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/histogram.cpp:160: error: (-215:Assertion failed) j < nimages in function 'histPrepareImages'


In [ ]:
for i in range(0,len(image_list)):

    for j in image_list[i]:
        print(j)
    